# Notebook 5: Model Compression
## Introduction
# Loads models from Notebook 3, compresses for h=6 (chosen for balance), evaluates trade-offs.
# Justification: Dynamic/Float16/Int quantization for LSTM; param reduction/feature selection for RF. Ensures sustainability (lower energy) while maintaining ~95% accuracy.


In [1]:
import pandas as pd
import tensorflow as tf
import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_absolute_error, mean_squared_error
import os
import time

chosen_h = 6
df = pd.read_csv('/content/drive/MyDrive/sus-lsa/featured_data.csv', index_col=0, parse_dates=True)
train_size = int(len(df) * 0.8)
train_df = df.iloc[:train_size]
test_df = df.iloc[train_size:]
features = [col for col in df.columns if col not in ['pm25_value', 'target']]
X_train = train_df[features]
y_train = train_df['target']
X_test = test_df[features]
y_test = test_df['target']

# LSTM Compression
lstm = tf.keras.models.load_model(f'/content/drive/MyDrive/sus-lsa/lstm_model_h{chosen_h}.h5', custom_objects={'mse': tf.keras.metrics.MeanSquaredError()})
# Dynamic Quant
converter = tf.lite.TFLiteConverter.from_keras_model(lstm)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = converter.convert()
with open(f'/content/drive/MyDrive/sus-lsa/lstm_dynamic_h{chosen_h}.tflite', 'wb') as f: f.write(tflite_model)
# Float16 Quant
converter = tf.lite.TFLiteConverter.from_keras_model(lstm)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
tflite_float16 = converter.convert()
with open(f'/content/drive/MyDrive/sus-lsa/lstm_float16_h{chosen_h}.tflite', 'wb') as f: f.write(tflite_float16)
# Int Quant (third for robustness)
converter = tf.lite.TFLiteConverter.from_keras_model(lstm)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
def representative_dataset():
    for _ in range(100): yield [X_test.sample(1).values.astype(np.float32)]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8, tf.lite.OpsSet.SELECT_TF_OPS]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_int = converter.convert()
with open(f'/content/drive/MyDrive/sus-lsa/lstm_int_h{chosen_h}.tflite', 'wb') as f: f.write(tflite_int)

# Evaluate (example for dynamic; repeat for others)
interpreter = tf.lite.Interpreter(model_path=f'/content/drive/MyDrive/sus-lsa/lstm_dynamic_h{chosen_h}.tflite')
interpreter.allocate_tensors()
preds = []
start = time.time()
for x in X_test.values.reshape(-1, 1, len(features)):
    interpreter.set_tensor(interpreter.get_input_details()[0]['index'], x.astype(np.float32))
    interpreter.invoke()
    preds.append(interpreter.get_tensor(interpreter.get_output_details()[0]['index']))
mae = mean_absolute_error(y_test, preds)
size = os.path.getsize(f'/content/drive/MyDrive/sus-lsa/lstm_dynamic_h{chosen_h}.tflite') / (1024*1024)
print(f'MAE: {mae}, Size: {size} MB, Time: {time.time() - start}s')

# RF Compression
rf = joblib.load(f'/content/drive/MyDrive/sus-lsa/rf_model_h{chosen_h}.pkl')
# Feature Selection
selector = SelectFromModel(rf, prefit=True)
X_train_sel = selector.transform(X_train)
X_test_sel = selector.transform(X_test)
rf_sel = RandomForestRegressor(n_estimators=50)  # Reduced estimators
rf_sel.fit(X_train_sel, y_train)
joblib.dump(rf_sel, f'/content/drive/MyDrive/sus-lsa/rf_sel_h{chosen_h}.pkl')
# Param Reduction
rf_red = RandomForestRegressor(n_estimators=50, max_depth=10)
rf_red.fit(X_train, y_train)
joblib.dump(rf_red, f'/content/drive/MyDrive/sus-lsa/rf_red_h{chosen_h}.pkl')
# Evaluate similarly

pd.DataFrame(compression_results).to_csv('/content/drive/MyDrive/sus-lsa/compression_results.csv')  # Justification: Compares size/accuracy trade-offs for sustainable AI.


Saved artifact at '/tmp/tmp0a972k12'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 1, 18), dtype=tf.float32, name='input_layer_2')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  137282919367888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137282919370768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137282919370960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137284215124240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137282919370384: TensorSpec(shape=(), dtype=tf.resource, name=None)


ConverterError: Could not translate MLIR to FlatBuffer.<unknown>:0: error: loc(callsite(callsite(fused["CudnnRNNV3:", "sequential_2_1/lstm_2_1/CudnnRNNV3@__inference_function_357"] at fused["StatefulPartitionedCall:", "StatefulPartitionedCall@__inference_signature_wrapper_388"]) at fused["StatefulPartitionedCall:", "StatefulPartitionedCall_1"])): 'tf.CudnnRNNV3' op is neither a custom op nor a flex op
<unknown>:0: note: loc(fused["StatefulPartitionedCall:", "StatefulPartitionedCall_1"]): called from
<unknown>:0: note: loc(callsite(callsite(fused["CudnnRNNV3:", "sequential_2_1/lstm_2_1/CudnnRNNV3@__inference_function_357"] at fused["StatefulPartitionedCall:", "StatefulPartitionedCall@__inference_signature_wrapper_388"]) at fused["StatefulPartitionedCall:", "StatefulPartitionedCall_1"])): see current operation: %1:5 = "tf.CudnnRNNV3"(%arg0, %arg1, %arg2, %arg3, %arg4) {T = f32, device = "", direction = "unidirectional", dropout = 0.000000e+00 : f32, input_mode = "linear_input", is_traini